In [1]:
# import libraries
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd

In [ ]:
# specify the url
urlpage = 'https://www.realestate.co.nz/residential/sale?by=featured&lct=r35&ql=20' 
print(urlpage)
# run firefox webdriver from executable path of your choice
driver = webdriver.Chrome(executable_path = 'C:\\Users\\User\\Desktop\\Quyen Nguyen\\Software\\chromedriver.exe')
# get web page
driver.get(urlpage)
# execute script to scroll down the page
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
# sleep for 30s
time.sleep(30)
# driver.quit()

https://www.realestate.co.nz/residential/sale?by=featured&lct=r35&ql=20


In [4]:
# find elements by xpath
results = driver.find_elements_by_class_name("listing-tile-title")
print('Number of results', len(results))
print(results[0].text)

Number of results 20
Promoted Private Lodge Style Country Residence


In [ ]:
# create empty array to store data
data = []
# loop over results
for result in results:
    product_name = result.text
    link = result.find_element_by_tag_name('a')
    product_link = link.get_attribute("href")
    data.append({ "product" : product_name, "link" : product_link})
    
data_df = pd.DataFrame(data)
print(data_df)